In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import numpy as np 
import pandas as pd 
import seaborn as sns
import warnings
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from catboost import CatBoostRegressor
import xgboost as xgb

In [ ]:
train=pd.read_csv('/kaggle/input/tabular-playground-series-mar-2022/train.csv')
test=pd.read_csv('/kaggle/input/tabular-playground-series-mar-2022/test.csv')
sub=pd.read_csv('/kaggle/input/tabular-playground-series-mar-2022/sample_submission.csv')

**STEP1:查看欄位資訊**

In [ ]:
#欄位資訊
print(train.info())
#前幾列資訊
print(train.head())
print(test.head())
print(sub.head())
#缺失值比例
print(train.isnull().sum()/len(train))
print(test.isnull().sum()/len(test))

In [ ]:
#處理時間資料
def new_time(df):
    df['time']=pd.to_datetime(df['time'])
    df['year']=df['time'].dt.year
    df['month']=df['time'].dt.month
    df['hour']=df['time'].dt.hour
    df['day']=df['time'].dt.day
    df['week']=df['time'].dt.week
    df['weekday']=df['time'].dt.weekday
    df['day_of_week']=df['time'].dt.day_name()
    return df

train=new_time(train)
test=new_time(test)
print(train.head())
print(train.tail())

**STEP2:簡單資料分析**

In [ ]:
#1方向統計
train['direction'].value_counts().plot(kind='bar')
plt.title('Direction counts')
plt.show()
#單方向(東南西北)的壅塞直較高

In [ ]:
#2壅塞值分布(機率分布圖)
sns.displot(train['congestion'])
plt.title('Congestion')
plt.show()

In [ ]:
#3星期與壅塞關系(直方圖)
train.groupby('day_of_week')['congestion'].sum().plot(kind='bar')
plt.xticks(rotation=45)
plt.title('Day vs Congestion')
plt.xlabel
plt.show()
#假日的壅塞狀況相比平日較不嚴重

In [ ]:
#4月份與壅塞的變化(折線圖)
train.groupby('month')['congestion'].sum().plot(marker='o')
plt.title('Month vs Congestion')
plt.show()

In [ ]:
#5每天時間對於壅塞的變化(折線圖)
train.groupby('hour')['congestion'].sum().plot(marker='o',color='r')
plt.title('Hour vs Congestion')
plt.show()

**STEP3:資料處理**

In [ ]:
#資料處理
target=train['congestion']
train.drop(['row_id','time','congestion','day_of_week'],axis=1,inplace=True)
test.drop(['row_id','time','day_of_week'],axis=1,inplace=True)
print(train.head())

#將類別特徵轉成向量
train = pd.get_dummies(train, dtype='float64')
test = pd.get_dummies(test, dtype='float64')
print(train.head())

**STEP4:套用模型**

In [ ]:
X_train,X_test,y_train,y_test = train_test_split(train,target, test_size=0.1,random_state=42)

#第一個MODEL:XGBOOST
xgb_model =xgb.XGBRegressor()
xgb_model.fit(X_train,y_train)
xgb_pred = xgb_model.predict(X_test)

#第二個MODEL:CATOOST
cat_model =CatBoostRegressor(random_state=42,
                            loss_function='RMSE',
                            eval_metric='RMSE',
                            silent=True)
cat_model.fit(X_train,y_train)
cat_pred = cat_model.predict(X_test)

In [ ]:
#比較RMSE
print('XGBOOST MSE:',mean_squared_error(y_test,xgb_pred))
print('CATBOOST MSE:',mean_squared_error(y_test,cat_pred))

In [ ]:
pred =cat_model.predict(test)
sub['congestion'] = pred
sub.to_csv('playground_sub2.csv', index=False)